In [1]:
import sys
import os
from glob import glob
sys.path.append("../../marineHeatWaves/")
sys.path.append("../analysis/physiology/")

import sys
import time
oldstderr = sys.stderr
sys.stderr = open('log.txt', 'w')


import ipywidgets as ipw

import sentinelsat
import satpy
from pyresample import create_area_def, AreaDefinition
from satpy.writers import compute_writer_results
import requests
from requests import auth, get

import xarray as xr

import warnings
warnings.filterwarnings("ignore")

import rasterio as rio

import pandas as pd
import tqdm
from multiprocessing import Pool

from cartopy import crs as ccrs
import seaborn as sns
import matplotlib.pyplot as plt

import tqdm

# coda_auth = auth.HTTPBasicAuth('tonycan', os.environ['ss_pass'])


from shapely import geometry, wkt

from dask.distributed import Client
from dask.diagnostics import ProgressBar
from dask import delayed
from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import cpu_count

from functools import partial
from itertools import zip_longest, cycle, chain

import gcsfs

In [ ]:
client = Client()

In [ ]:
os.environ['ss_pass'];

In [2]:
mp_pool = Pool(cpu_count() - 1)

# Whole Record, One Isolate Sentinel 3 Validation

I think I'm going to take the entire Sentinel 3 Record over a single spot and look at the relationship between chlorophyll + performance throughout that period, perhaps punctuated by s3?

The general question here is whether chlorophyll-a patterns can be explained by performance, where MHWs are a particular subset of "

In [3]:
plankton = pd.read_csv("../data/Phytoplankton_temperature_growth_rate_dataset_2016_01_29/traits_derived_2016_01_29.csv", engine='python')

plankton = plankton[
    (plankton.habitat == 'marine') & 
    (plankton.curvequal == 'good')
]


We'll only use the NE-Pacific isolate (#240)

In [3]:
chosen_isolate = plankton[plankton['isolate.code'] == 240]

## Acquire Entire Sentinel-3 Record for this Region

In [ ]:
s3_api_historical = sentinelsat.SentinelAPI(
    'tonycan', 
    os.environ['ss_pass'], 
    api_url = 'https://codarep.eumetsat.int/'
)
s3_api_recent = sentinelsat.SentinelAPI(
    'tonycan', 
    os.environ['ss_pass'], 
    api_url = 'https://coda.eumetsat.int/'
)


In [ ]:
isolate_mhw_wkt = wkt.dumps(
    geometry.Polygon.from_bounds(
        *geometry.Point([
            chosen_isolate['isolation.longitude'],
            chosen_isolate['isolation.latitude']
        ]).buffer(2).bounds
    )
)

In [ ]:
queryParams = dict(
    area = isolate_mhw_wkt, 
    producttype='OL_2_WRR___',
    date = ('20160101', '20191231')
)

qr_historical = s3_api_historical.query(**queryParams)
qr_recent = s3_api_recent.query(**queryParams)

In [ ]:
s3_result_combined = pd.concat([
    s3_api_historical.to_geodataframe(qr_historical),
    s3_api_recent.to_geodataframe(qr_recent)
])

In [ ]:
ax = plt.axes()
[ax.axvline(x.beginposition) for _, x in s3_result_combined.iterrows()];


Clearly there's a data gap in 2018, I can't really explain this. Let's just do the historical data from `codarep`. 

In [ ]:
s3_result_historical = s3_api_historical.to_geodataframe(qr_historical)

In [4]:
def download_image(image, path, auth):
    _, image = image
    r = get(image.link, auth=auth)
    exitpath = os.path.join(path, f'{image.title}.zip')
    with open(exitpath, 'wb') as f:
        f.write(r.content)
    return(exitpath)

In [5]:
download_location = '/tmp/nepacs3/historical/'
os.makedirs(download_location, exist_ok=True)
download_func = partial(download_image, path=download_location, auth=coda_auth)

NameError: name 'coda_auth' is not defined

In [ ]:
mp_pool.map(download_func, s3_result_historical.iterrows())

**Unzip all of the data**:

In [ ]:
%%bash -s "$download_location"
ls ${1}*.zip | parallel -t unzip  > /dev/null

## Process `chl_oc4me` and `chl_nn` data

**Note**: I initially tried to run the following code in parallel in this notebook, but ran into issues with dask/satpy locality and pickling rasterio objects. Spent the better part of a day working that out, and couldn't figure it out. The final solution was to break the `reproject_and_save` function out into its own module (`reproject_s3.py`) and ran that using `gnu parallel` as follows: 

    find . -name "*.SEN3" -type d | parallel --no-notice -k  "python /home/ec2-user/mhw_stressviz/validation/reproject_s3.py {}"
    
I will keep the old stuff here for posterity + reference. 

In [ ]:
# def reproject_and_save(img_directory, projection=ccrs.Mercator()):
#     chl_data = ['chl_oc4me', 'chl_nn']

#     _sc = satpy.Scene(glob(img_directory+"/*") , reader='olci_l2', )

#     _sc.load(chl_data)

#     outpath = os.path.join(img_directory, f"{_sc.attrs['start_time']:%Y%m%d}.cf")

#     _sc_resampled = _sc.resample(
#             _sc[chl_data[0]].attrs['area'].compute_optimal_bb_area(projection.proj4_params)
#     )
    
# return(_sc_resampled.save_datasets(writer='cf', filename=outpath, compute=False))

    
# #     delayed = []
# #     for d in chl_data:
# #         _sc_resampled = _sc.resample(
# #             _sc[d].attrs['area'].compute_optimal_bb_area(projection.proj4_params)
# #         )
# #         outpath = os.path.join(img_directory, f"{d}-{_sc.attrs['start_time']:%Y%m%d}.cf")
        
# #         delayed.append(_sc_resampled.save_dataset(d, writer='cf', filename=outpath, compute=False))



In [10]:
def reproject_save_v2(path):
    dss_dask = []
    dss = ['chl_oc4me', 'chl_nn']
    try:
        _sc = satpy.Scene(glob(path+"/*") , reader='olci_l2')
        _sc.load(dss)
    except Exception as e:
        return
        
    rs_sc = _sc.resample(_sc['chl_nn'].area.compute_optimal_bb_area(ccrs.Mercator().proj4_params))
    for i in dss:
        outpath = os.path.join(path, f"{i}-{_sc.attrs['start_time']:%Y%m%d}.tif")
        dss_dask.append(rs_sc.save_dataset(i, outpath, writer='geotiff', compute=False))
    return(dss_dask)
        
        


## Reproject and Clip all outputs

In [ ]:
processed_files = glob(os.path.join(download_location, "*/*.tif"))

In [ ]:
dates = list(map(get_date_from_file, processed_files))

In [ ]:
pd.to_datetime(dates).sort_values()

In [ ]:
datasets = []
for file in processed_files:
    try:
        datasets.append(rio.open(file))
    except Exception as e:
        datasets.append(None)
        

In [ ]:
datasets